In [1]:
import pandas as pd

# Supongamos que tu archivo CSV se llama 'libros.csv'
df = pd.read_csv('C:\\Users\\Fukushima\\Documents\\GitHub\\ProyectoFinalVV\\data\\raw\\librosyafiltrados.csv')


In [2]:
import scipy.sparse as sp
from joblib import load

matrix_sparse = load('C:\\Users\\Fukushima\\Documents\\GitHub\\ProyectoFinalVV\\src\\model.pkl')


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy.sparse import hstack

features = df[['Rating', 'pagesNumber', 'Authors']]

# Normalizar las columnas numéricas (Rating y pagesNumber)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features[['Rating', 'pagesNumber']])

# Realizar one-hot encoding para la columna 'Authors'
encoder = OneHotEncoder()
authors_encoded = encoder.fit_transform(features[['Authors']])

# Combinar las características escaladas y el encoding de autores en una matriz dispersa
matrix_features = hstack([features_scaled, authors_encoded])

# Asegurarse de que matrix_features sea una matriz bidimensional
print(matrix_features.shape) 


(1060441, 362084)


In [5]:
from scipy.sparse import coo_matrix

# Supongamos que 'matrix_features' es una coo_matrix
coo_matrix = matrix_features

# Acceder a datos en formato COO
data = coo_matrix.data
rows = coo_matrix.row
cols = coo_matrix.col



In [4]:
from sklearn.neighbors import NearestNeighbors

# Entrena el modelo KNN con la matriz dispersa
model_knn = NearestNeighbors(n_neighbors=6, algorithm="brute", metric="cosine")
model_knn.fit(matrix_features)



NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [6]:
def get_book_recommendations(book_title):
    # Encuentra el índice del libro en el DataFrame
    book_index = df[df["Name"] == book_title].index[0]

    # Extrae la fila correspondiente del DataFrame como una matriz dispersa
    book_vector = matrix_features[book_index]

    # Encuentra las distancias y los índices de los libros más similares
    distances, indices = model_knn.kneighbors(book_vector.reshape(1, -1))

    # Crear una lista de libros recomendados con sus distancias
    similar_books = [(df["Name"].iloc[i], distances[0][j]) for j, i in enumerate(indices[0])]

    return similar_books[1:]  # Excluir el libro en sí mismo



In [7]:
input_book = "Harry Potter and the Order of the Phoenix"
recommendations = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for book, distance in recommendations:
    print("- Book: {}, Distance: {:.4f}".format(book, distance))


TypeError: 'coo_matrix' object is not subscriptable

In [15]:
from scipy.sparse import coo_matrix

# Verifica el tipo de la matriz
print(type(matrix_features))


<class 'scipy.sparse._coo.coo_matrix'>


otro metodo 

Primero pasé   las columnas numéricas en una sola columna y luego escalé los valores númericas para que no haya un número que demonie al otro

In [8]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Seleccionar columnas numéricas relevantes
numerical_columns = ['pagesNumber', 'Rating', 'RatingDistTotal_numeric']  

if all(col in df.columns for col in numerical_columns):
    numerical_features = df[numerical_columns].fillna(0).values  # Asegurarse de que no haya NaNs
    
    # Escalar los valores numéricos para evitar que un rango domine a los otros
    scaler = StandardScaler()
    numerical_features = scaler.fit_transform(numerical_features)
else:
    print("Algunas columnas numéricas faltan en el DataFrame.")
    numerical_features = np.array([])


In [9]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def is_probably_same_book(title1, title2, author1, author2):
    title1 = title1.lower()
    title2 = title2.lower()
    
    common_keywords = ["annotated", "&", "and"]
    if author1 == author2:
        for word in common_keywords:
            if word in title1 and word in title2:
                return True
        return similar(title1, title2) > 0.85
    else:
        return similar(title1, title2) > 0.9

def get_title_similarity(input_title, all_titles):
    similarities = [similar(input_title, title) for title in all_titles]
    return np.array(similarities)

def get_book_recommendations(input_book, num_recommendations=10):
    try:
        idx = df[df['Name'] == input_book].index[0]
    except IndexError:
        print(f"Libro '{input_book}' no encontrado en el DataFrame.")
        return [], []
    
    input_title = df['Name'].iloc[idx]
    input_author = df['Authors'].iloc[idx].lower()
    title_similarities = get_title_similarity(input_title, df['Name'].values)
    
    input_features = numerical_features[idx].reshape(1, -1)
    numerical_similarities = cosine_similarity(input_features, numerical_features)[0]

    total_similarity = (0.7 * title_similarities) + (0.3 * numerical_similarities)
    
    sim_scores = list(enumerate(total_similarity))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:]  # Excluir el libro de entrada

    filtered_scores = []
    seen_titles = set()
    for i, score in sim_scores:
        candidate_title = df['Name'].iloc[i]
        candidate_author = df['Authors'].iloc[i].lower()
        
        if candidate_title not in seen_titles and not is_probably_same_book(input_title, candidate_title, input_author, candidate_author):
            filtered_scores.append((i, score))
            seen_titles.add(candidate_title)
            if len(filtered_scores) == num_recommendations:
                break
    
    book_indices = [i[0] for i in filtered_scores]
    return df[['Name', 'Authors']].iloc[book_indices], filtered_scores

In [12]:
# Probar las recomendaciones
input_book = "Harry Potter and the Prisoner of Azkaban"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))
    

Book recommendations for 'Harry Potter and the Prisoner of Azkaban':
- Book: Harry Potter et le prisonnier d'Azkaban, Author: J.K. Rowling, Similarity Score: 0.8494
- Book: Harry Potter e il prigioniero di Azkaban, Author: J.K. Rowling, Similarity Score: 0.8250
- Book: Harry Potter y el prisionero de Azkaban (Harry Potter, #3), Author: J.K. Rowling, Similarity Score: 0.7571
- Book: Harry Potter et le Prisonnier d'Azkaban (Harry Potter, #3), Author: J.K. Rowling, Similarity Score: 0.7571
- Book: Harry Potter Y El Prisionero De Azkaban (Harry Potter, #3), Author: J.K. Rowling, Similarity Score: 0.7571
- Book: Harry Potter e o Prisioneiro de Azkaban (Harry Potter, #3), Author: J.K. Rowling, Similarity Score: 0.7571
- Book: Harry Potter et le prisonnier d'Azkaban (Harry Potter, #3), Author: J.K. Rowling, Similarity Score: 0.7429
- Book: Harry Potter e il prigioniero di Azkaban (Harry Potter, #3), Author: J.K. Rowling, Similarity Score: 0.7242
- Book: Harri Potter maen yr Athronydd, Author:

In [ ]:
# Probar las recomendaciones
input_book = "Pride and Prejudice"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))


Book recommendations for 'Pride and Prejudice':
- Book: The Annotated Pride & Prejudice, Author: Jane Austen, Similarity Score: 0.7480
- Book: War and Peace, Author: Leo Tolstoy, Similarity Score: 0.7370
- Book: Hide and Seek, Author: James Patterson, Similarity Score: 0.7273
- Book: Krieg und Frieden, Author: Leo Tolstoy, Similarity Score: 0.7216
- Book: Krieg Und Frieden, Author: Leo Tolstoy, Similarity Score: 0.7212
- Book: Krieg und Frieden., Author: Leo Tolstoy, Similarity Score: 0.7042
- Book: Le Poids De La Preuve, Author: Scott Turow, Similarity Score: 0.6989
- Book: War And Peace, Author: Leo Tolstoy, Similarity Score: 0.6932
- Book: Wall and Piece, Author: Banksy, Similarity Score: 0.6711
- Book: Orgullo y Prejuicio, Author: Jane Austen, Similarity Score: 0.6684


In [ ]:
# Probar las recomendaciones
input_book = "Sense and Sensibility"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))

Book recommendations for 'Sense and Sensibility':
- Book: Sentido y sensibilidad, Author: Jane Austen, Similarity Score: 0.6907
- Book: Fathers and Sons, Author: Ivan Turgenev, Similarity Score: 0.6613
- Book: Peter Pan and Wendy, Author: J.M. Barrie, Similarity Score: 0.6500
- Book: Feuer und Stein, Author: Diana Gabaldon, Similarity Score: 0.6496
- Book: Sein und Zeit, Author: Martin Heidegger, Similarity Score: 0.6410
- Book: Romeo and Juliet, Author: William Shakespeare, Similarity Score: 0.6403
- Book: Brideshead Revisited, Author: Evelyn Waugh, Similarity Score: 0.6376
- Book: A Great and Terrible Beauty, Author: Libba Bray, Similarity Score: 0.6352
- Book: The Horse and His Boy, Author: C.S. Lewis, Similarity Score: 0.6307
- Book: Secret Garden, Author: Frances Hodgson Burnett, Similarity Score: 0.6294


In [ ]:
# Probar las recomendaciones
input_book = "The Catcher in the Rye"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))

Book recommendations for 'The Catcher in the Rye':
- Book: The Cat in the Hat, Author: Dr. Seuss, Similarity Score: 0.8237
- Book: The Catcher in the Rye / キャッチャー・イン・ザ・ライ, Author: J.D. Salinger, Similarity Score: 0.8049
- Book: The Baron in the Trees, Author: Italo Calvino, Similarity Score: 0.7741
- Book: The Name of the Rose, Author: Umberto Eco, Similarity Score: 0.7656
- Book: The Lady in the Lake, Author: Raymond Chandler, Similarity Score: 0.7535
- Book: The Name Of The Rose, Author: Umberto Eco, Similarity Score: 0.7323
- Book: Death on the Nile, Author: Agatha Christie, Similarity Score: 0.7283
- Book: The Catcher in the Rye: Annotations and Study Aids, Author: J.D. Salinger, Similarity Score: 0.7278
- Book: Cat in the Hat, Author: Dr. Seuss, Similarity Score: 0.7264
- Book: The Mote in God's Eye, Author: Larry Niven, Similarity Score: 0.7149


In [ ]:
# Probar las recomendaciones
input_book = "The Count of Monte Cristo"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))

Book recommendations for 'The Count of Monte Cristo':
- Book: The Count of Monte Cristo (sch Cl), Author: Alexandre Dumas, Similarity Score: 0.8932
- Book: Le Comte de Monte-Cristo, Author: Alexandre Dumas, Similarity Score: 0.8143
- Book: Il Conte di Montecristo, Author: Alexandre Dumas, Similarity Score: 0.7958
- Book: El Conde de Montecristo, Author: Alexandre Dumas, Similarity Score: 0.7666
- Book: The House of the Spirits, Author: Isabel Allende, Similarity Score: 0.7526
- Book: The Hound of the Baskervilles, Author: Arthur Conan Doyle, Similarity Score: 0.7407
- Book: The Lord of the Rings, Author: J.R.R. Tolkien, Similarity Score: 0.7234
- Book: The Colour of Magic, Author: Terry Pratchett, Similarity Score: 0.7115
- Book: The Day of the Triffids, Author: John Wyndham, Similarity Score: 0.7037
- Book: The Cask of Amontillado, Author: Edgar Allan Poe, Similarity Score: 0.7015


In [ ]:
input_book = "The Hound of the Baskervilles"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))

Book recommendations for 'The Hound of the Baskervilles':
- Book: The Hound of the Baskervilles: Easy Readers, Author: Arthur Conan Doyle, Similarity Score: 0.8639
- Book: Hound of the Baskervilles Minibook, Author: Arthur Conan Doyle, Similarity Score: 0.8551
- Book: Der Hund Von Baskerville, Author: Arthur Conan Doyle, Similarity Score: 0.8283
- Book: Der Hund von Baskerville, Author: Arthur Conan Doyle, Similarity Score: 0.8283
- Book: Drama Scripts: Hound of the Baskervilles, Author: Arthur Conan Doyle, Similarity Score: 0.8275
- Book: The Hound of the Baskervilles (Sherlock Holmes, #5), Author: Arthur Conan Doyle, Similarity Score: 0.8075
- Book: Tess of the D'Urbervilles, Author: Thomas Hardy, Similarity Score: 0.7518
- Book: Tess of the d'Urbervilles, Author: Thomas Hardy, Similarity Score: 0.7516
- Book: Tess of the D'urbervilles, Author: Thomas Hardy, Similarity Score: 0.7513
- Book: The Hound of the Baskervilles: Level 3: 3,100 Word Vocabulary, Author: Arthur Conan Doyle, Sim

In [ ]:
input_book = "At the Mountains of Madness"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))

Book recommendations for 'At the Mountains of Madness':
- Book: The Fountains of Paradise, Author: Arthur C. Clarke, Similarity Score: 0.7818
- Book: The Mountain of Names, Author: Alex Shoumatoff, Similarity Score: 0.7629
- Book: Monkey Mountain Madness, Author: Jeanne Phillips, Similarity Score: 0.7525
- Book: Fountains of Paradise, Author: Arthur C. Clarke, Similarity Score: 0.7334
- Book: A Mountain of Blintzes, Author: Barbara Diamond Goldin, Similarity Score: 0.7277
- Book: The Comforts of Madness, Author: Paul Sayer, Similarity Score: 0.7191
- Book: A Moment's Madness, Author: Helen Kirkman, Similarity Score: 0.7141
- Book: The Mountain Cat Murders, Author: Rex Stout, Similarity Score: 0.7131
- Book: The Monuments of Mars, Author: Richard C. Hoagland, Similarity Score: 0.7053
- Book: The Four Foundations of Mindfulness, Author: U. Silananda, Similarity Score: 0.6963


In [ ]:
input_book = "The Outsider"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))

Book recommendations for 'The Outsider':
- Book: The Outlanders, Author: Ashley Carter, Similarity Score: 0.7969
- Book: Maxwell: The Outsider, Author: Tom Bower, Similarity Score: 0.7878
- Book: Step Outside, Author: Peter London, Similarity Score: 0.7711
- Book: The Rider, Author: Tim Krabbé, Similarity Score: 0.7642
- Book: Sister Outsider, Author: Audre Lorde, Similarity Score: 0.7575
- Book: The Founders, Author: Aaron Fletcher, Similarity Score: 0.7529
- Book: The Curse, Author: Adam Cleveland, Similarity Score: 0.7518
- Book: The Backslider, Author: Levi S. Peterson, Similarity Score: 0.7508
- Book: The Hustler, Author: John Henry Mackay, Similarity Score: 0.7474
- Book: The Executioners, Author: Phil Robin Clarke, Similarity Score: 0.7447


In [ ]:
input_book = "The Lord of the Rings"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))

Book recommendations for 'The Lord of the Rings':
- Book: The Tao Te Ching, Author: Lao Tzu, Similarity Score: 0.6999
- Book: The Illustrated Book of Five Rings, Author: Miyamoto Musashi, Similarity Score: 0.6994
- Book: La Compagnia dell'Anello (The Lord of the Rings, #1), Author: J.R.R. Tolkien, Similarity Score: 0.6986
- Book: The Call of the Wild; White Fang, Author: Jack London, Similarity Score: 0.6960
- Book: Le Seigneur des Anneaux (The Lord of the Rings, #1-3), Author: J.R.R. Tolkien, Similarity Score: 0.6960
- Book: Le Seigneur des Anneaux (The Lord of the Rings, #1-3), Author: J.R.R. Tolkien, Similarity Score: 0.6960
- Book: True Story of the Three Little Pigs, Author: Jon Scieszka, Similarity Score: 0.6959
- Book: Il Signore degli Anelli (The Lord of the Rings, #1-3), Author: J.R.R. Tolkien, Similarity Score: 0.6958
- Book: Il signore degli anelli (The Lord of the Rings, #1-3), Author: J.R.R. Tolkien, Similarity Score: 0.6958
- Book: El señor de los anillos (The Lord of the

In [ ]:
input_book = "1984"
recommended_books, scores = get_book_recommendations(input_book)

print("Book recommendations for '{}':".format(input_book))
for (book, author), (index, score) in zip(recommended_books.values, scores):
    print("- Book: {}, Author: {}, Similarity Score: {:.4f}".format(book, author, score))

Book recommendations for '1984':
- Book: Animal Farm / 1984, Author: George Orwell, Similarity Score: 0.5365
- Book: 1974, Author: David Peace, Similarity Score: 0.5171
- Book: 1974, Author: David Peace, Similarity Score: 0.5170
- Book: 1945, Author: Newt Gingrich, Similarity Score: 0.5078
- Book: 1945, Author: Newt Gingrich, Similarity Score: 0.5078
- Book: 1934, Author: Alberto Moravia, Similarity Score: 0.5047
- Book: 1988, Author: Andrew McGahan, Similarity Score: 0.5026
- Book: Fever 1793, Author: Laurie Halse Anderson, Similarity Score: 0.4833
- Book: 1942, Author: Robert Conroy, Similarity Score: 0.4833
- Book: 1945, Author: Robert Conroy, Similarity Score: 0.4832
- Book: Fever 1793, Author: Laurie Halse Anderson, Similarity Score: 0.4830
- Book: Fever 1793, Author: Laurie Halse Anderson, Similarity Score: 0.4829
- Book: Fever 1793, Author: Laurie Halse Anderson, Similarity Score: 0.4813
- Book: 1776, Author: David McCullough, Similarity Score: 0.4749
- Book: 1776, Author: David

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
